In [1]:
!pip install transformers datasets evaluate


In [2]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [3]:
!pip install -U fsspec==2023.6.0


In [36]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Reduce to small subset
train_data = dataset["train"].select(range(1000))
val_data = dataset["validation"].select(range(300))


In [37]:
print(train_data[0])

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [41]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [43]:
def preprocess_function(batch):
    inputs = ["summarize: " + doc for doc in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"], max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [44]:
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [49]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",            # Still works
    run_name="t5-summarization-run"
)


In [46]:
!pip install -U transformers


In [50]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer
)


/tmp/ipython-input-50-1738218431.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


Step,Training Loss
10,11.355200
20,11.481100
30,11.115900
40,10.525000
50,9.566800
60,7.893200
70,6.869300
80,5.432000
90,4.035200
100,2.935400


Step,Training Loss
10,11.355200
20,11.481100
30,11.115900
40,10.525000
50,9.566800
60,7.893200
70,6.869300
80,5.432000
90,4.035200
100,2.935400


In [52]:
eval_results = trainer.evaluate()
print(eval_results)


{'eval_loss': 0.7027572393417358, 'eval_runtime': 226.7883, 'eval_samples_per_second': 1.323, 'eval_steps_per_second': 0.168, 'epoch': 3.0}


In [53]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

test_text = """
The Apollo missions were a series of space flights undertaken by NASA to land humans on the Moon and bring them back safely to Earth. The most famous of these missions, Apollo 11, successfully landed the first humans, Neil Armstrong and Buzz Aldrin, on the lunar surface on July 20, 1969.
"""

summary = summarizer("summarize: " + test_text, max_length=50, min_length=25, do_sample=False)
print("Summary:", summary[0]['summary_text'])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Summary: Apollo 11 successfully landed the first humans, Neil Armstrong and Buzz Aldrin, on the lunar surface on July 20, 1969 .


In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)


Device set to use cpu


In [5]:
test_texts = [
    """India became the first country to land a spacecraft near the Moon’s south pole...""",
    """The stock market saw a significant rise today, with major indices showing positive growth...""",
    """Lionel Messi won his 8th Ballon d'Or after leading Argentina to World Cup victory..."""
]

for i, text in enumerate(test_texts):
    summary = summarizer("summarize: " + text, max_length=50, min_length=25, do_sample=False)
    print(f"\nExample {i+1} Summary:", summary[0]['summary_text'])


Your max_length is set to 50, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 50, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Example 1 Summary: the first country to land a spacecraft near the moon’s south pole . it is the first spacecraft in the world to land in the u.s.


Your max_length is set to 50, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Example 2 Summary: the stock market saw a significant rise today, with major indices showing positive growth . the stock exchange saw an impressive rise today with a major rise in indexes .

Example 3 Summary: Lionel Messi won his 8th ballon d'Or after leading argentina to world cup victory . lionel messi led argentine to the world cup .


In [6]:
test_texts = [
    "India became the first country to land a spacecraft near the Moon’s south pole...",
    "The stock market saw a significant rise today...",
    "Lionel Messi won his 8th Ballon d'Or..."
]


In [7]:
for i, text in enumerate(test_texts):
    summary = summarizer("summarize: " + text, max_length=50, min_length=25, do_sample=False)
    print(f"\nExample {i+1} Summary:", summary[0]['summary_text'])


Your max_length is set to 50, but your input_length is only 24. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 50, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Example 1 Summary: the first country to land a spacecraft near the moon’s south pole . it is the first spacecraft in the world to land in the u.s.


Your max_length is set to 50, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Example 2 Summary: the stock market saw a significant rise today . a rise in the market is expected to be expected in the next few weeks .

Example 3 Summary: Lionel Messi won his 8th ballon d'Or . he won his eighth ballon...


In [8]:
model.save_pretrained("t5_summarizer_model")
tokenizer.save_pretrained("t5_summarizer_model")


('t5_summarizer_model/tokenizer_config.json',
 't5_summarizer_model/special_tokens_map.json',
 't5_summarizer_model/spiece.model',
 't5_summarizer_model/added_tokens.json')